In [1]:
import numpy as np 
import pandas as pd
import torch
from torch import nn

# Data preparation

In [2]:
import pickle

# Load
with open('Data/nfl_train_test_data.pkl', 'rb') as f:
    data = pickle.load(f)
    X_train = data['X_train']
    Y_train = data['Y_train']
    X_test = data['X_test']
    Y_test = data['Y_test']

In [3]:
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Initialize the lists for the normalized data
X_train_normalized_list = []
Y_train_normalized_list = []
X_test_normalized_list = []
Y_test_normalized_list = []

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Normalize the training data
for x, y in zip(X_train, Y_train):
    # Flatten the tensors along the frame and player dimensions
    x_flattened = x.view(x.shape[0] * x.shape[1], -1).numpy()
    y_flattened = y.view(y.shape[0] * y.shape[1], -1).numpy()

    # Fit the scalers to the flattened tensors and transform
    x_normalized_np = scaler_x.fit_transform(x_flattened)
    y_normalized_np = scaler_y.fit_transform(y_flattened)

    # Convert the normalized arrays back to tensors and reshape to original shape
    x_normalized = torch.tensor(x_normalized_np, dtype=torch.float32).view(x.shape)
    y_normalized = torch.tensor(y_normalized_np, dtype=torch.float32).view(y.shape)

    # Append to the lists
    X_train_normalized_list.append(x_normalized)
    Y_train_normalized_list.append(y_normalized)

# Normalize the testing data
for x, y in zip(X_test, Y_test):
    # Flatten the tensors along the frame and player dimensions
    x_flattened = x.view(x.shape[0] * x.shape[1], -1).numpy()
    y_flattened = y.view(y.shape[0] * y.shape[1], -1).numpy()

    # Transform the flattened tensors using the fitted scalers
    x_normalized_np = scaler_x.transform(x_flattened)
    y_normalized_np = scaler_y.transform(y_flattened)

    # Convert the normalized arrays back to tensors and reshape to original shape
    x_normalized = torch.tensor(x_normalized_np, dtype=torch.float32).view(x.shape)
    y_normalized = torch.tensor(y_normalized_np, dtype=torch.float32).view(y.shape)

    # Append to the lists
    X_test_normalized_list.append(x_normalized)
    Y_test_normalized_list.append(y_normalized)

# Split the normalized lists into training and validation sets
X_train_val, X_val, Y_train_val, Y_val = train_test_split(X_train_normalized_list, Y_train_normalized_list, test_size=0.2, random_state=42)

# Create datasets and DataLoaders for train, validation, and test sets
train_dataset = TensorDataset(torch.cat(X_train_val, dim=0), torch.cat(Y_train_val, dim=0))
val_dataset = TensorDataset(torch.cat(X_val, dim=0), torch.cat(Y_val, dim=0))
test_dataset = TensorDataset(torch.cat(X_test_normalized_list, dim=0), torch.cat(Y_test_normalized_list, dim=0))

# Increase batch size if your hardware allows
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# Training 

## LSTM

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import time
from Scripts.Models.LSTM import LSTMModel

input_size = X_train_normalized_list[0].shape[2]
output_size = Y_train_normalized_list[0].shape[2]
hidden_size = 128
num_layers = 1
bidirectional = True
epochs = 10

model = LSTMModel(input_size, hidden_size, output_size, num_layers, bidirectional)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=3, verbose=True)

clip = 1  # threshold for gradient clipping

# Add early stopping
patience = 5
no_improvement = 0
best_val_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time()
    model.train()
    train_loss = 0.0
    for x_train_batch, y_train_batch in zip(X_train_normalized_list, Y_train_normalized_list):
        optimizer.zero_grad()
        output = model(x_train_batch)
        loss = criterion(output, y_train_batch)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(X_train_normalized_list)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_val_batch, y_val_batch in zip(X_val, Y_val):
            output = model(x_val_batch)
            loss = criterion(output, y_val_batch)
            val_loss += loss.item()

    val_loss /= len(X_val)

    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {time.time() - start_time:.4f}s')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        no_improvement += 1
        if no_improvement >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

    scheduler.step(val_loss)

model.eval()

C:\Users\spman\AppData\Roaming\Python\Python311\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/10, Train Loss: 0.3154, Val Loss: 0.2501, Time: 50.3904s
Epoch 2/10, Train Loss: 0.2504, Val Loss: 0.2399, Time: 41.9247s
Epoch 3/10, Train Loss: 0.2441, Val Loss: 0.2372, Time: 44.4626s
Epoch 4/10, Train Loss: 0.2415, Val Loss: 0.2355, Time: 41.9359s
Epoch 5/10, Train Loss: 0.2400, Val Loss: 0.2343, Time: 49.0352s
Epoch 6/10, Train Loss: 0.2389, Val Loss: 0.2334, Time: 44.6976s
Epoch 7/10, Train Loss: 0.2381, Val Loss: 0.2327, Time: 45.4100s
Epoch 8/10, Train Loss: 0.2374, Val Loss: 0.2322, Time: 47.3448s
Epoch 9/10, Train Loss: 0.2369, Val Loss: 0.2318, Time: 52.2790s
Epoch 10/10, Train Loss: 0.2365, Val Loss: 0.2315, Time: 43.7505s


LSTMModel(
  (lstm): LSTM(12, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=12, bias=True)
)

## SocialLSTM

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import time
from Scripts.Models.SocialLSTM import SocialLSTM

input_size = X_train_normalized_list[0].shape[2]
output_size = Y_train_normalized_list[0].shape[2]
hidden_size = 128
num_layers = 1
bidirectional = True
epochs = 10

model = SocialLSTM(input_size, hidden_size, output_size, num_layers, bidirectional)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=3, verbose=True)

clip = 1  # threshold for gradient clipping

# Add early stopping
patience = 5
no_improvement = 0
best_val_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time()
    model.train()
    train_loss = 0.0
    for x_train_batch, y_train_batch in zip(X_train_normalized_list, Y_train_normalized_list):
        optimizer.zero_grad()
        output = model(x_train_batch)
        loss = criterion(output, y_train_batch)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(X_train_normalized_list)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_val_batch, y_val_batch in zip(X_val, Y_val):
            output = model(x_val_batch)
            loss = criterion(output, y_val_batch)
            val_loss += loss.item()

    val_loss /= len(X_val)

    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {time.time() - start_time:.4f}s')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        no_improvement += 1
        if no_improvement >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

    scheduler.step(val_loss)

model.eval()

C:\Users\spman\AppData\Roaming\Python\Python311\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


IndexError: too many indices for tensor of dimension 3

## SocialVAE

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import time
from Scripts.Models.SocialVAE import SocialVAE

input_size = X_train_normalized_list[0].shape[2]
output_size = Y_train_normalized_list[0].shape[2]
hidden_size = 128
latent_size = 64
num_layers = 2
bidirectional = True
epochs = 10

model = SocialVAE(input_size, hidden_size, latent_size, output_size, num_layers, bidirectional)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=3, verbose=True)

clip = 1  # threshold for gradient clipping

# Add early stopping
patience = 5
no_improvement = 0
best_val_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time()
    model.train()
    train_loss = 0.0
    for x_train, y_train in zip(X_train_normalized_list, Y_train_normalized_list):
        optimizer.zero_grad()
        recon_x, mu, log_var = model(x_train)
        recon_loss = criterion(recon_x, y_train)
        kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        kld_loss /= x_train.shape[0] * x_train.shape[1]
        loss = recon_loss + kld_loss
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(X_train_normalized_list)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_val_batch, y_val_batch in zip(X_val, Y_val):
            recon_x, mu, log_var = model(x_val_batch)
            recon_loss = criterion(recon_x, x_val_batch)
            kld_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
            kld_loss /= x_val_batch.shape[0] * x_val_batch.shape[1]
            loss = recon_loss + kld_loss
            val_loss += loss.item()

    val_loss /= len(X_val)

    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {time.time() - start_time:.4f}s')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        no_improvement += 1
        if no_improvement >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

    scheduler.step(val_loss)


C:\Users\spman\AppData\Roaming\Python\Python311\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Input shape: torch.Size([299, 23, 12])
After LSTM: torch.Size([299, 23, 128])


IndexError: too many indices for tensor of dimension 2

## Diffusion Transformer

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import time
from Scripts.Models.Transformer import DiffusionTransformer

input_size = 12
output_size = 12
hidden_size = 128
latent_size = 64
num_layers = 2
dropout = 0.1
d_ff = 512
max_len = 5000
epochs = 10

model = DiffusionTransformer(d_model=input_size, nhead=4, num_encoder_layers=num_layers, d_ff=d_ff, dropout=dropout, max_len=max_len)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=3, verbose=True)

clip = 1  # threshold for gradient clipping

# Add early stopping
patience = 5
no_improvement = 0
best_val_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time()
    model.train()
    train_loss = 0.0
    for x_train, y_train in zip(X_train_normalized_list, Y_train_normalized_list):
        optimizer.zero_grad()
        output = model(x_train)
        loss = criterion(output, y_train)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(X_train_normalized_list)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_val_batch, y_val_batch in zip(X_val, Y_val):
            output = model(x_val_batch)
            loss = criterion(output, y_val_batch)
            val_loss += loss.item()

    val_loss /= len(X_val)

    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {time.time() - start_time:.4f}s')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        no_improvement += 1
        if no_improvement >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

    scheduler.step(val_loss)

C:\Users\spman\AppData\Roaming\Python\Python311\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/10, Train Loss: 0.4934, Val Loss: 0.3463, Time: 64.5290s
Epoch 2/10, Train Loss: 0.3922, Val Loss: 0.3227, Time: 68.4109s
Epoch 3/10, Train Loss: 0.3729, Val Loss: 0.3155, Time: 64.1057s
Epoch 4/10, Train Loss: 0.3626, Val Loss: 0.3108, Time: 60.3002s
Epoch 5/10, Train Loss: 0.3559, Val Loss: 0.3073, Time: 62.7075s
Epoch 6/10, Train Loss: 0.3507, Val Loss: 0.3052, Time: 63.2961s
Epoch 7/10, Train Loss: 0.3469, Val Loss: 0.3019, Time: 59.1792s
Epoch 8/10, Train Loss: 0.3434, Val Loss: 0.3003, Time: 61.2998s
Epoch 9/10, Train Loss: 0.3407, Val Loss: 0.2985, Time: 58.4982s
Epoch 10/10, Train Loss: 0.3380, Val Loss: 0.2962, Time: 60.5739s
